<a href="https://colab.research.google.com/github/marsgav/TowardTacklingEuphemisms/blob/main/Parrot_Paraphraser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Martha Gavidia
Tackling Euphemisms in NLP

This notebook contains code for paraphrasing sample PETs for the paraphrasing stage in "Toward Tackling Euphemisms in NLP"

This uses the Parrot Paraphraser paraphrasing framework found here:+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

In [ ]:
pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

  Cloning https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to /tmp/pip-req-build-envq67ks
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git /tmp/pip-req-build-envq67ks
     |████████████████████████████████| 4.0 MB 8.9 MB/s 
     |████████████████████████████████| 1.2 MB 46.2 MB/s 
     |████████████████████████████████| 50 kB 3.6 MB/s 
     |████████████████████████████████| 79 kB 3.9 MB/s 
     |████████████████████████████████| 77 kB 1.2 MB/s 
     |████████████████████████████████| 596 kB 46.6 MB/s 
     |████████████████████████████████| 6.6 MB 28.0 MB/s 
     |████████████████████████████████| 880 kB 46.3 MB/s 
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8695 sha256=022141a3a3ee7a1eefc221c85a8b636795a4717a5aba8a33623887f73bc67d43
  Stored in directory: /tmp/pip-ephem-wheel-cache-atd5a5jo/wheels/ff/27/79/a53e92f32da44c8c4f51094ca156e6ff91103b7e48a341d62b
  Created wheel for python-Levenshtein: file

In [ ]:
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

In [ ]:
#this file contains the euphemisms from our dataset that were found in the concrete phrases dataset, 46 total
import pandas as pd
df = pd.read_csv('concrete_euphemisms.csv')
euphs = df["Expression"].to_list()
phrases = euphs

In [ ]:
#first attempt- basic implementation
paraphrases = {}
for i, row in df['Expression'].iteritems():
  para_phrases = parrot.augment(input_phrase=str(row),
                                diversity_ranker="levenshtein",
                                do_diverse=True, 
                                max_return_phrases = 30, 
                                max_length=32, 
                                adequacy_threshold = 0.5, 
                                fluency_threshold = 0.40)
  for para_phrase in para_phrases:
    if para_phrase == row:
      with open("parrot_output.csv", "a") as f:
        print(row, para_phrase, file = f)

In [ ]:
df2 = pd.read_csv('parrot_output.csv',sep= '\(|,|,', header= None, index_col= False,)
df2.columns = ['phrase', 'paraphrase', 'quality_score']
df2['quality_score'] = df2['quality_score'].str.replace('\)', '')
#df2['paraphrase'] = df2['paraphrase'].str.replace("( [^a-zA-Z ]+|#|'|,)", "")
#df2 = df2.groupby(['phrase'])['paraphrase'].apply(', '.join).reset_index()
df2['quality_score'] = df2['quality_score'].astype(int)
df2 = df2[df2['quality_score'] > 12]


In [ ]:
df2.to_csv('concreteeuphs_paraphrase_filtered.csv')

In [ ]:
df3 = pd.read_csv('concreteeuphs_paraphrase_filtered.csv')
df3 = df3.groupby(['phrase'])['paraphrase'].apply(', '.join).reset_index()
df3

In [ ]:
df3

,phrase,paraphrase
0,able-bodied,"'able', 'well-managed', 'healthy', 'bodied', '..."
1,between jobs,"'between the jobs', 'between work'"
2,broken home,"'broken house', 'home broken', 'broken home?',..."
3,capital punishment,"'death penalty', 'the death penalty', 'capital..."
4,collateral damage,"'damages', 'damage', 'compensation for damages..."
5,comfort women,"'comfort the ladies', 'women', 'comfort ladies..."
6,correctional facility,"'facility of correction', 'correctional instit..."
7,detention camp,"'camp', 'detention center', 'the detention cen..."
8,developing country,"'country in development', 'a developing nation..."
9,drinking problem,"'problem with alcohol', 'problem with drinking..."
